In [15]:
from azure.cosmos import CosmosClient
from dotenv import load_dotenv
import json
from openai import AzureOpenAI
import os

def generate_openai_embeddings(input_text):
    client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version = "2025-01-01-preview",
    azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
    )

    response = client.embeddings.create(
        input = input_text,
        model= "text-embedding-ada-002"
    )

    return response.data[0].embedding

cosmos_url =  os.getenv("COSMOS_ENDPOINT")
cosmos_key =  os.getenv("COSMOS_KEY")
cosmos_database_name = "linkedin_data"
cosmos_container_name = "linkedin_data_vector"

cosmos_client = CosmosClient(cosmos_url, cosmos_key)
cosmos_container = cosmos_client.get_database_client(cosmos_database_name).get_container_client(cosmos_container_name)

question = "List all employees working at M1."

query_embedding = generate_openai_embeddings(question)

top_n = 10

query = """
    SELECT c.full_name, c.full_name_url, c.role, c.company, VectorDistance(c.request_vector, @query_embedding) AS SimilarityScore
    FROM c
    """
parameters = [
    {"name": "@query_embedding", "value": query_embedding},
]

result = list(cosmos_container.query_items(
    query=query,
    parameters=parameters,
    enable_cross_partition_query=True
))

results = sorted(result, key=lambda x: x['SimilarityScore'], reverse=True)[:top_n]
results

[{'full_name': 'ChunJin Hoo',
  'full_name_url': 'https://www.linkedin.com/sales/lead/ACwAAArT3ysBNEMGyxNYvUyLyZts0t7BfBcF1Xg,NAME_SEARCH,cQjN?_ntb=2l7hMCeUQFKi%2Br3KmkAhjQ%3D%3D',
  'role': 'General Manager, Core Network Planning (Packet Core & RSP)',
  'company': 'M1 Limited',
  'SimilarityScore': 0.8329576145114843},
 {'full_name': 'Dinesh Krishnarajah',
  'full_name_url': 'https://www.linkedin.com/sales/lead/ACwAAAEAPQAB9ikP04sNz__sAuLmX1FFVXrhu48,NAME_SEARCH,vLvo?_ntb=2l7hMCeUQFKi%2Br3KmkAhjQ%3D%3D',
  'role': 'Viewed\nYou’ve already seen Dinesh Krishnarajah’s profile before.',
  'company': 'M1 Limited',
  'SimilarityScore': 0.8329576145114843},
 {'full_name': 'Denis Seek',
  'full_name_url': 'https://www.linkedin.com/sales/lead/ACwAAAGrZHIBczqmqwcT-YAMtHpWQWbxTjNM5qI,NAME_SEARCH,QVLt?_ntb=2l7hMCeUQFKi%2Br3KmkAhjQ%3D%3D',
  'role': 'Chief Technical Officer',
  'company': 'M1 Limited',
  'SimilarityScore': 0.8329576145114843},
 {'full_name': 'Hasini K',
  'full_name_url': 'https://